In [1]:
import pandas as pd
from myfun.addcol import addcolumns
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
import joblib
import json
import os

In [3]:
model_folder_name = "train_use_full_data_update_weather_data"  #訓練好的權重  來自train異地同時.ipynb
with open(f"./model_each_location/{model_folder_name}/d.json", "r", encoding="utf-8") as file:
    mae_data = json.load(file)

In [4]:
#讀入json檔案  必且將每個模型的mae由小到大依序排序
for k in mae_data.keys():
    mae_data[k] = dict(sorted(mae_data[k].items(), key=lambda item: item[1]))
mae_data

{'02': {'02_07.pkl': 131.63387368015486,
  '02_03.pkl': 148.9494026377359,
  '02_01.pkl': 157.29159529658952,
  '02_06.pkl': 164.8974798982957,
  '02_04.pkl': 178.3880294357141,
  '02_05.pkl': 236.92519867868288,
  '02_14.pkl': 250.27164768944252,
  '02_13.pkl': 271.7814698951735,
  '02_10.pkl': 296.6597617956016,
  '02_09.pkl': 364.5657067406608,
  '02_11.pkl': 486.5527985457143,
  '02_08.pkl': 490.6013231639061,
  '02_12.pkl': 500.5751417951683,
  '02_16.pkl': 602.7255868558245,
  '02_15.pkl': 653.4253487692202,
  '02_17.pkl': 707.270496562365},
 '03': {'03_02.pkl': 161.0008668940303,
  '03_04.pkl': 189.5358006131556,
  '03_01.pkl': 189.660406351199,
  '03_06.pkl': 227.4046924454917,
  '03_07.pkl': 234.48687958859145,
  '03_05.pkl': 264.339052888098,
  '03_10.pkl': 314.98827932300026,
  '03_13.pkl': 364.1884834303437,
  '03_14.pkl': 379.6113030335456,
  '03_11.pkl': 466.2711881442685,
  '03_09.pkl': 476.82371276837563,
  '03_12.pkl': 566.1586075627325,
  '03_08.pkl': 612.192990029394

In [5]:
def combine_weather(df1:pd.DataFrame):  #定義函數連結中央氣象局花蓮站的觀測資料
    weather_df = pd.read_csv("../資料前處理以及第三方資料/10mins_weather_data.csv")
    weather_df["DateTime"] =  pd.to_datetime(weather_df["DateTime"])
    weather_df["日期"] = pd.to_datetime(weather_df["DateTime"].dt.date)
    weather_df["時間"] = weather_df["DateTime"].dt.hour.astype(str).str.zfill(2)
    weather_df["時段"] = weather_df["DateTime"].dt.minute.astype(str).str.zfill(2)
    
    weather_df.fillna(0,inplace=True)
    df = pd.merge(
        df1,
        weather_df,
        left_on=["日期","時間","時段"],
        right_on=["日期","時間","時段"],
        how="left",
        suffixes=["","_r"]
    )

    return(df)
#將不同觀測站的資料load進來並放在字典裡
df_dict = {}
for i in range(1,18):
    location = "0" + str(i) if i<10 else str(i)
    df = pd.read_csv(f"./train_combine_groupby/{location}_traingroupby.csv")
    df = addcolumns(df)
    df = combine_weather(df)
    df_dict[location] = df

In [6]:
#將不同模型load進來並放在字典裡
model_dict = {}
for k1 in mae_data.keys():
    for model_name in mae_data[k1].keys():
        model_file_path = os.path.join("./model_each_location",model_folder_name,model_name)
        model_dict[model_name] = joblib.load(model_file_path)

In [7]:
#將檔案讀進來  新增日期時間位置等欄位以便我們找資料對應
DataName = "./36_TestSet_SubmissionTemplate/upload(no answer).csv"
ans = pd.read_csv(DataName, encoding='utf-8')
ans["序號"] = ans["序號"].astype(str)
ans["日期"] = pd.to_datetime(ans["序號"].str.slice(0,8))
ans["時間"] =ans["序號"].str.slice(8,10)
ans["時段"] =ans["序號"].str.slice(10,12)
ans["位置"] =ans["序號"].str.slice(12,14)
ans["模型順位"] = None
ans["使用模型MAE"] = None
for i in range(len(ans)):
    date,time,timecount,ans_location = ans.loc[i,"日期"],ans.loc[i,"時間"],ans.loc[i,"時段"],ans.loc[i,"位置"]  #找出答案該個row的時期時間時段以及位置
    mae_table = mae_data[ans_location]   #找出這個位置的所有模型(來自另外16個模型) 並且已按照MAE順序排
    j = 1
    for model_name in mae_table.keys():  #從MAE最小的模型(觀測站)去找有沒有該觀測站的資料  如果有就用該資料跟模型來進行predict
        X_location = model_name[3:5]     # X_locatioan是訓練集模型的地點
        df_sametimedata = df_dict[X_location]
        search_df = df_sametimedata.loc[(df_sametimedata["日期"]==date) & (df_sametimedata["時間"]==time) & (df_sametimedata["時段"]==timecount),]
        if len(search_df)==1:  #如果有找資料 就用這個模型predict 並且break迴圈  沒有就繼續往MAE較大的模型找 找到都沒有則答案欄位就保持NA
            X_test = search_df.loc[:,["WindSpeed(m/s)","Pressure(hpa)", "Temperature(°C)","Humidity(%)","Sunlight(Lux)",
                            'SunshineDuration', 'UVIndex',
                            'AirTemperature', 'DailyExtremeHighAirTemperature',
                            'DailyExtremeLowAirTemperature', 'RelativeHumidity',
                            'Power(mW)']]  
            X_test = X_test.rename(columns=lambda col: f"{col}_x")  #將每個變數都加上_x以符合train那邊的形式
            model = model_dict[model_name]
            predictions = model.predict(X_test)
            ans.loc[i,"答案"] = predictions[0]  
            ans.loc[i,"模型順位"] = j  #以下資訊供調整參數參考
            ans.loc[i,"使用模型MAE"] = mae_table[model_name]
            break
        j = j+1 
    if i %100==0:
        print(i)

            

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500


In [8]:
# 有些日期沒有異地同時的資料  該日期對應預測結果會有NA，故改用第三方天氣數據訓練出來的預測的模型
ans_nona = ans.loc[~ans["答案"].isna(),].reset_index(drop=True)
ans_na = ans.loc[ans["答案"].isna(),].reset_index(drop=True)
ans_nona.loc[ans["使用模型MAE"]>=700,]

,序號,答案,日期,時間,時段,位置,模型順位,使用模型MAE


In [9]:
model_lastday_folder_name = "train_use_full_data_update_weather_datasingle_location"  # 來自train同地同時_10mins天氣資料.ipynb
with open(f"./model_each_location/{model_lastday_folder_name}/d.json", "r", encoding="utf-8") as file:
    mae_lastday_data = json.load(file)

In [10]:
model_lastday_dict = {}
for k1 in mae_lastday_data.keys():
    model_name = k1 
    model_file_path = os.path.join("./model_each_location",model_lastday_folder_name,model_name)
    model_lastday_dict[k1] = joblib.load(model_file_path)

In [11]:
# 若答案為NA代表沒有異地同時之資料可以預測，故改採用同地同時的模型
for i in range(len(ans_na)):  
    if i%10==0:
        print(i)
    time,timecount,ans_location = ans_na.loc[i,"時間"],ans_na.loc[i,"時段"],ans_na.loc[i,"位置"]
    df_sametimedata = df_dict[ans_location]
    df_sametimedata = addcolumns(df_sametimedata)
    df_sametimedata = combine_weather(df_sametimedata)
    days = 1
    while True:
        date = ans_na.loc[i,"日期"] - pd.Timedelta(days=days)
        search_df = df_sametimedata.loc[(df_sametimedata["日期"]==date) & (df_sametimedata["時間"]==time) & (df_sametimedata["時段"]==timecount),]
        if len(search_df)==1:
            X_test = search_df.loc[:,[
                            'SunshineDuration', 'UVIndex',
                            'AirTemperature', 'DailyExtremeHighAirTemperature',
                            'DailyExtremeLowAirTemperature', 'RelativeHumidity']]  
            model = model_lastday_dict[f"{ans_location}_weather.pkl"]
            predictions = model.predict(X_test)
            ans_na.loc[i,"答案"] = predictions[0]
            break
        days = days + 1

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


In [13]:
res = pd.concat([ans_nona,ans_na],axis=0).reset_index(drop=True)
# res.loc[:,["序號","答案"]].to_csv("1128ans3_alldata_udate_weather.csv",index=False) #最高的答案